# Transfer learning tests
Eamon Whalen

In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.models.pointRegressorSurrogateModel import PointRegressor
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load test data

In [2]:
# dataDir = "data/2D_Truss_v1.3/conmech/"
# testDir = os.path.join(dataDir, 'design_7_N_1000/')
# allGraphsUnfiltered = loadConmechGraphs(testDir)

# dataDir = "data/endLoadsv1.0/conmech/"
# testDir = os.path.join(dataDir, 'design_7_N_1000/')
# allGraphsUnfiltered = loadConmechGraphs(testDir)

dataDir = "data/tower1.0/conmech/"
testDir = os.path.join(dataDir, 'design_5_N_1000/')
allGraphsUnfiltered = loadConmechGraphs(testDir, loadDims=[0])

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.027434
std,0.005930
min,0.022934
25%,0.023780
50%,0.025250
75%,0.028404
max,0.065064


In [3]:
testData = filterbyDisp(allGraphsUnfiltered, 0.9)
maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in testData]
source = pd.DataFrame(maxes, columns=['maxes'])
maxDispCutoff = source.max().item()
source.describe()

,maxes
count,900.000000
mean,0.025836
std,0.002804
min,0.022934
25%,0.023687
50%,0.024775
75%,0.027176
max,0.034477


## 2. Load pre-train data

In [4]:
pretrainDirs = glob.glob('data/2D_Truss_v1.3/conmech/design_7_N_1000/')

# pretrainDirs = glob.glob(os.path.join(dataDir, '*1000/'))
# pretrainDirs.remove(testDir)

allPretrainGraphs = []
for pretrainDir in pretrainDirs:
    designName = pretrainDir.split('/')[-2].split('_N')[0]
    print(f'loading from {pretrainDir}')
    graphsUnfiltered = loadConmechGraphs(pretrainDir)
    graphs = filterbyDisp(graphsUnfiltered, 0.9)
    allPretrainGraphs.extend(graphs)

print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

loading from data/2D_Truss_v1.3/conmech/design_7_N_1000/
loaded 900 pretraining graphs


## 3. Initial training

In [5]:
saveDir = 'results/transferLrn_des7_tower5_01/'
epochs = 100
ptrGcn = FeaStNet()
history = ptrGcn.trainModel(pretrainData, pretrainValData, 
                            epochs=epochs,
                            saveDir=saveDir+f'preTrain/gcn/')

ptrGcnCheckptFile = ptrGcn.checkptFile
plotHistory(history)

train model: flatten: True
train model: self.flatten: True
fitSS: self.flatten: True
epoch: 0   trainLoss: 9.3096e-01   valLoss:8.6974e-01  time: 2.16e+00
epoch: 1   trainLoss: 6.8632e-01   valLoss:8.5991e-01  time: 1.77e+00
epoch: 2   trainLoss: 5.3786e-01   valLoss:9.4513e-01  time: 1.78e+00
epoch: 3   trainLoss: 4.4543e-01   valLoss:1.7662e+00  time: 1.79e+00
epoch: 4   trainLoss: 3.6750e-01   valLoss:4.4537e+00  time: 1.77e+00
epoch: 5   trainLoss: 3.0831e-01   valLoss:8.1462e+00  time: 1.77e+00
epoch: 6   trainLoss: 2.5948e-01   valLoss:9.3518e+00  time: 1.76e+00
epoch: 7   trainLoss: 2.3203e-01   valLoss:5.0187e+00  time: 1.76e+00
epoch: 8   trainLoss: 1.9679e-01   valLoss:2.5170e+00  time: 1.78e+00
epoch: 9   trainLoss: 1.8080e-01   valLoss:1.2414e+00  time: 1.78e+00
epoch: 10   trainLoss: 1.6973e-01   valLoss:1.0225e+00  time: 1.77e+00
epoch: 11   trainLoss: 1.4591e-01   valLoss:6.2598e-01  time: 1.78e+00
epoch: 12   trainLoss: 1.4978e-01   valLoss:7.4784e-01  time: 1.76e+00
ep

alt.Chart(...)

In [6]:
trainRes = ptrGcn.testModel(pretrainData)
testRes = ptrGcn.testModel(testData) # unseen topology
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000011,0.001990,0.100537,7.403459e-01,0.949667,7.878571e-01,0.000000e+00
test,1.470075,0.867565,0.966870,-1.687669e+06,0.000000,-3.368448e+06,-1.241156e+07


## 4. Transfer learning study

In [7]:
# trainDataDirs = glob.glob(os.path.join(dataDir, 'design_7*/'))
trainDataDirs = glob.glob(os.path.join(dataDir, '*/'))
trainDataDirs.remove(testDir)

allResults = []
for trainDataDir in trainDataDirs:
    trainDataUnfiltered = loadConmechGraphs(trainDataDir)
    trainData = filterbyDispValue(trainDataUnfiltered, maxDispCutoff)
    trainSize = len(trainData)
    print(f'reading from {trainDataDir}')
    print(f'loaded train set of size {trainSize}')
    
    
    ### fresh neural network ###
    gcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                         epochs=epochs, 
                         saveDir=saveDir+f'{trainSize:05}/gcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Fresh'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Fresh'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)

    
    ### transfer learning ###
    ptrGcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                             restartFile=ptrGcnCheckptFile,
                             epochs=epochs, 
                             saveDir=saveDir+f'{trainSize:05}/ptrGcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Transfer learning'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Transfer learning'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
    ### random forest ###
    rf = PointRegressor('Random Forest')
    rf.trainModel(trainData, trainData, 
                  useXFeatures=False,
                  saveDir=saveDir+f'{trainSize:05}/rf/')

    trainRes = rf.testModel(trainData)
    trainRes['Model'] = 'Random Forest'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = rf.testModel(testData)
    testRes['Model'] = 'Random Forest'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
pd.DataFrame(allResults)

reading from data/tower1.0/conmech/design_5_N_200/
loaded train set of size 180
train model: flatten: True
train model: self.flatten: True
fitSS: self.flatten: True
epoch: 0   trainLoss: 8.0157e-01   valLoss:9.0456e-01  time: 1.43e+00
epoch: 1   trainLoss: 5.2345e-01   valLoss:8.9783e-01  time: 1.43e+00
epoch: 2   trainLoss: 3.4602e-01   valLoss:8.8903e-01  time: 1.42e+00
epoch: 3   trainLoss: 2.3784e-01   valLoss:8.7072e-01  time: 1.39e+00
epoch: 4   trainLoss: 1.5705e-01   valLoss:8.5016e-01  time: 1.44e+00
epoch: 5   trainLoss: 1.1786e-01   valLoss:8.3249e-01  time: 1.46e+00
epoch: 6   trainLoss: 9.4067e-02   valLoss:8.2349e-01  time: 1.43e+00
epoch: 7   trainLoss: 7.7052e-02   valLoss:8.1461e-01  time: 1.42e+00
epoch: 8   trainLoss: 7.1516e-02   valLoss:7.9984e-01  time: 1.38e+00
epoch: 9   trainLoss: 7.0579e-02   valLoss:7.4647e-01  time: 1.38e+00
epoch: 10   trainLoss: 6.8106e-02   valLoss:6.6452e-01  time: 1.39e+00
epoch: 11   trainLoss: 6.2419e-02   valLoss:6.3430e-01  time: 1.

epoch: 12   trainLoss: 6.2915e-02   valLoss:7.1970e-02  time: 1.39e+00
epoch: 13   trainLoss: 5.8072e-02   valLoss:6.9960e-02  time: 1.38e+00
epoch: 14   trainLoss: 5.5700e-02   valLoss:7.1944e-02  time: 1.39e+00
epoch: 15   trainLoss: 5.3955e-02   valLoss:6.9553e-02  time: 1.43e+00
epoch: 16   trainLoss: 5.1224e-02   valLoss:6.2372e-02  time: 1.43e+00
epoch: 17   trainLoss: 4.8053e-02   valLoss:5.4488e-02  time: 1.42e+00
epoch: 18   trainLoss: 4.5773e-02   valLoss:4.8779e-02  time: 1.43e+00
epoch: 19   trainLoss: 4.4404e-02   valLoss:4.5629e-02  time: 1.43e+00
epoch: 20   trainLoss: 4.3002e-02   valLoss:4.4109e-02  time: 1.40e+00
epoch: 21   trainLoss: 4.1030e-02   valLoss:4.3841e-02  time: 1.41e+00
epoch: 22   trainLoss: 3.8961e-02   valLoss:4.4307e-02  time: 1.41e+00
epoch: 23   trainLoss: 3.7310e-02   valLoss:4.4409e-02  time: 1.41e+00
epoch: 24   trainLoss: 3.5989e-02   valLoss:4.3476e-02  time: 1.42e+00
epoch: 25   trainLoss: 3.4650e-02   valLoss:4.1400e-02  time: 1.43e+00
epoch:

epoch: 25   trainLoss: 9.4180e-03   valLoss:3.1269e-02  time: 3.53e+00
epoch: 26   trainLoss: 7.9601e-03   valLoss:3.0167e-02  time: 3.52e+00
epoch: 27   trainLoss: 8.3642e-03   valLoss:2.7795e-02  time: 3.54e+00
epoch: 28   trainLoss: 6.1520e-03   valLoss:2.6368e-02  time: 3.56e+00
epoch: 29   trainLoss: 4.3842e-03   valLoss:2.1943e-02  time: 3.54e+00
epoch: 30   trainLoss: 3.7459e-03   valLoss:2.4606e-02  time: 3.55e+00
epoch: 31   trainLoss: 2.9943e-03   valLoss:1.8426e-02  time: 3.52e+00
epoch: 32   trainLoss: 2.7931e-03   valLoss:1.6684e-02  time: 3.54e+00
epoch: 33   trainLoss: 2.8996e-03   valLoss:1.8241e-02  time: 3.53e+00
epoch: 34   trainLoss: 2.1274e-03   valLoss:1.5234e-02  time: 3.52e+00
epoch: 35   trainLoss: 1.6969e-03   valLoss:1.3791e-02  time: 3.53e+00
epoch: 36   trainLoss: 1.4995e-03   valLoss:1.2894e-02  time: 3.61e+00
epoch: 37   trainLoss: 1.9659e-03   valLoss:1.2898e-02  time: 3.55e+00
epoch: 38   trainLoss: 2.2946e-03   valLoss:1.8662e-02  time: 3.53e+00
epoch:

epoch: 39   trainLoss: 3.0557e-03   valLoss:4.2022e-03  time: 3.50e+00
epoch: 40   trainLoss: 2.7812e-03   valLoss:4.3370e-03  time: 3.39e+00
epoch: 41   trainLoss: 1.2637e-03   valLoss:2.2937e-03  time: 3.36e+00
epoch: 42   trainLoss: 1.9341e-03   valLoss:2.8682e-03  time: 3.39e+00
epoch: 43   trainLoss: 1.0900e-03   valLoss:3.8998e-03  time: 3.46e+00
epoch: 44   trainLoss: 1.0719e-03   valLoss:2.2800e-03  time: 3.47e+00
epoch: 45   trainLoss: 1.0674e-03   valLoss:2.3108e-03  time: 3.49e+00
epoch: 46   trainLoss: 9.6114e-04   valLoss:3.6627e-03  time: 3.48e+00
epoch: 47   trainLoss: 1.1081e-03   valLoss:2.3981e-03  time: 3.47e+00
epoch: 48   trainLoss: 7.6299e-04   valLoss:1.9374e-03  time: 3.52e+00
epoch: 49   trainLoss: 6.7079e-04   valLoss:2.5439e-03  time: 3.48e+00
epoch: 50   trainLoss: 6.4509e-04   valLoss:2.5374e-03  time: 3.48e+00
epoch: 51   trainLoss: 5.8198e-04   valLoss:2.1978e-03  time: 3.55e+00
epoch: 52   trainLoss: 1.1200e-03   valLoss:2.2416e-03  time: 3.46e+00
epoch:

epoch: 54   trainLoss: 3.0039e-02   valLoss:1.2635e-01  time: 8.66e-02
epoch: 55   trainLoss: 2.9980e-02   valLoss:1.2272e-01  time: 8.67e-02
epoch: 56   trainLoss: 3.0481e-02   valLoss:1.2233e-01  time: 8.70e-02
epoch: 57   trainLoss: 3.0366e-02   valLoss:1.2433e-01  time: 8.72e-02
epoch: 58   trainLoss: 2.8833e-02   valLoss:1.2264e-01  time: 8.67e-02
epoch: 59   trainLoss: 2.9182e-02   valLoss:1.1842e-01  time: 8.78e-02
epoch: 60   trainLoss: 2.8352e-02   valLoss:1.1662e-01  time: 8.73e-02
epoch: 61   trainLoss: 2.8113e-02   valLoss:1.1972e-01  time: 8.90e-02
epoch: 62   trainLoss: 2.7261e-02   valLoss:1.2143e-01  time: 9.07e-02
epoch: 63   trainLoss: 2.6778e-02   valLoss:1.1943e-01  time: 9.15e-02
epoch: 64   trainLoss: 2.6885e-02   valLoss:1.2168e-01  time: 9.18e-02
epoch: 65   trainLoss: 2.7143e-02   valLoss:1.1702e-01  time: 8.96e-02
epoch: 66   trainLoss: 2.7102e-02   valLoss:1.1962e-01  time: 8.88e-02
epoch: 67   trainLoss: 2.7318e-02   valLoss:1.1849e-01  time: 8.93e-02
epoch:

...

epoch: 50   trainLoss: 1.3361e-03   valLoss:2.6183e-02  time: 7.09e+00
epoch: 51   trainLoss: 1.2734e-03   valLoss:2.2011e-02  time: 7.11e+00
epoch: 52   trainLoss: 2.5431e-03   valLoss:2.1636e-02  time: 7.14e+00
epoch: 53   trainLoss: 1.2726e-03   valLoss:1.5585e-02  time: 7.20e+00
epoch: 54   trainLoss: 1.4002e-03   valLoss:1.3951e-02  time: 7.11e+00
epoch: 55   trainLoss: 2.5334e-03   valLoss:1.4936e-02  time: 7.10e+00
epoch: 56   trainLoss: 1.5687e-03   valLoss:1.7395e-02  time: 7.06e+00
epoch: 57   trainLoss: 2.0158e-03   valLoss:2.2874e-02  time: 7.03e+00
epoch: 58   trainLoss: 1.1628e-03   valLoss:2.3256e-02  time: 7.11e+00
epoch: 59   trainLoss: 1.2983e-03   valLoss:1.9713e-02  time: 7.12e+00
epoch: 60   trainLoss: 1.5794e-03   valLoss:1.7587e-02  time: 7.07e+00
epoch: 61   trainLoss: 1.8241e-03   valLoss:1.8205e-02  time: 7.13e+00
epoch: 62   trainLoss: 3.0274e-03   valLoss:1.6327e-02  time: 7.13e+00
epoch: 63   trainLoss: 1.7494e-03   valLoss:1.5605e-02  time: 7.08e+00
epoch:

epoch: 64   trainLoss: 5.7664e-04   valLoss:2.9955e-03  time: 7.13e+00
epoch: 65   trainLoss: 1.1024e-03   valLoss:2.0612e-03  time: 7.04e+00
epoch: 66   trainLoss: 1.2813e-03   valLoss:4.7559e-03  time: 7.11e+00
epoch: 67   trainLoss: 1.0686e-03   valLoss:3.3877e-03  time: 7.13e+00
epoch: 68   trainLoss: 7.0961e-04   valLoss:2.9002e-03  time: 7.19e+00
epoch: 69   trainLoss: 7.5887e-04   valLoss:2.4624e-03  time: 7.07e+00
epoch: 70   trainLoss: 8.8057e-04   valLoss:3.2198e-03  time: 7.04e+00
epoch: 71   trainLoss: 1.3809e-03   valLoss:3.0058e-03  time: 7.08e+00
epoch: 72   trainLoss: 1.1400e-03   valLoss:3.4794e-03  time: 7.05e+00
epoch: 73   trainLoss: 7.1897e-04   valLoss:2.5653e-03  time: 7.11e+00
epoch: 74   trainLoss: 5.8114e-04   valLoss:2.3366e-03  time: 7.19e+00
epoch: 75   trainLoss: 5.3605e-04   valLoss:3.5257e-03  time: 7.21e+00
epoch: 76   trainLoss: 6.7255e-04   valLoss:5.0574e-03  time: 7.12e+00
epoch: 77   trainLoss: 8.6647e-04   valLoss:4.2512e-03  time: 7.17e+00
epoch:

epoch: 78   trainLoss: 2.1546e-03   valLoss:6.6875e-02  time: 1.65e-01
epoch: 79   trainLoss: 2.7053e-03   valLoss:5.8923e-02  time: 1.62e-01
epoch: 80   trainLoss: 2.1536e-03   valLoss:5.4863e-02  time: 1.64e-01
epoch: 81   trainLoss: 1.5685e-03   valLoss:5.3319e-02  time: 1.65e-01
epoch: 82   trainLoss: 1.8736e-03   valLoss:4.8656e-02  time: 1.62e-01
epoch: 83   trainLoss: 1.5453e-03   valLoss:4.6031e-02  time: 1.63e-01
epoch: 84   trainLoss: 1.6466e-03   valLoss:4.6693e-02  time: 1.63e-01
epoch: 85   trainLoss: 8.6944e-04   valLoss:5.0016e-02  time: 1.65e-01
epoch: 86   trainLoss: 1.1344e-03   valLoss:4.9260e-02  time: 1.64e-01
epoch: 87   trainLoss: 8.6539e-04   valLoss:4.8484e-02  time: 1.68e-01
epoch: 88   trainLoss: 1.1156e-03   valLoss:4.8509e-02  time: 1.60e-01
epoch: 89   trainLoss: 7.8766e-04   valLoss:4.5456e-02  time: 1.65e-01
epoch: 90   trainLoss: 8.0288e-04   valLoss:3.9761e-02  time: 1.61e-01
epoch: 91   trainLoss: 5.9060e-04   valLoss:3.8091e-02  time: 1.61e-01
epoch:

epoch: 92   trainLoss: 5.4118e-03   valLoss:1.2118e-02  time: 1.57e-01
epoch: 93   trainLoss: 4.5079e-03   valLoss:1.1905e-02  time: 1.57e-01
epoch: 94   trainLoss: 4.6104e-03   valLoss:1.1016e-02  time: 1.62e-01
epoch: 95   trainLoss: 4.4962e-03   valLoss:1.0728e-02  time: 1.59e-01
epoch: 96   trainLoss: 3.9407e-03   valLoss:1.2022e-02  time: 1.60e-01
epoch: 97   trainLoss: 3.6472e-03   valLoss:1.3584e-02  time: 1.58e-01
epoch: 98   trainLoss: 3.4581e-03   valLoss:1.4598e-02  time: 1.59e-01
epoch: 99   trainLoss: 3.1927e-03   valLoss:1.4142e-02  time: 1.58e-01
loading checkpoint 84
trained 24 random forest models in 2.76 seconds


,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,2.660634e-06,0.001215,0.079675,0.673650,0.576182,-13.553525,-108.045485,Fresh,Train,180
1,1.031741e-03,0.022531,0.481587,-755.943496,0.000000,-1824.017334,-9924.245758,Fresh,Test,180
2,1.884177e-07,0.000335,0.021375,0.947683,0.979292,0.370694,-0.839688,Transfer learning,Train,180
3,1.220445e-05,0.002801,0.163308,-3.028061,0.000000,-57.299070,-276.002546,Transfer learning,Test,180
4,3.212383e-09,0.000028,0.001506,0.998414,1.000000,0.997109,0.991977,Random Forest,Train,180
5,2.297604e-08,0.000073,0.003909,0.989085,1.000000,0.981748,0.959730,Random Forest,Test,180
6,4.916766e-07,0.000520,0.031258,0.837433,0.841597,-1.160613,-18.464501,Fresh,Train,445
7,1.824250e-04,0.010227,0.338595,-107.680914,0.000000,-410.452558,-2639.883435,Fresh,Test,445
8,1.581865e-07,0.000303,0.019524,0.951290,0.965992,0.459569,-1.157768,Transfer learning,Train,445
9,1.574461e-05,0.003146,0.196118,0.295736,0.383904,-72.745692,-407.906966,Transfer learning,Test,445


In [8]:
# df = pd.DataFrame(allResults)
df = pd.read_csv('results/transferLrn_des7_tower5_01/testResults.csv')
df

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,2.660634e-06,0.001215,0.079675,0.673650,0.576182,-13.553525,-108.045485,Fresh,Train,180
1,1.031741e-03,0.022531,0.481587,-755.943496,0.000000,-1824.017334,-9924.245758,Fresh,Test,180
2,1.884177e-07,0.000335,0.021375,0.947683,0.979292,0.370694,-0.839688,Transfer learning,Train,180
3,1.220445e-05,0.002801,0.163308,-3.028061,0.000000,-57.299070,-276.002546,Transfer learning,Test,180
4,3.212383e-09,0.000028,0.001506,0.998414,1.000000,0.997109,0.991977,Random Forest,Train,180
5,2.297604e-08,0.000073,0.003909,0.989085,1.000000,0.981748,0.959730,Random Forest,Test,180
6,4.916766e-07,0.000520,0.031258,0.837433,0.841597,-1.160613,-18.464501,Fresh,Train,445
7,1.824250e-04,0.010227,0.338595,-107.680914,0.000000,-410.452558,-2639.883435,Fresh,Test,445
8,1.581865e-07,0.000303,0.019524,0.951290,0.965992,0.459569,-1.157768,Transfer learning,Train,445
9,1.574461e-05,0.003146,0.196118,0.295736,0.383904,-72.745692,-407.906966,Transfer learning,Test,445


In [12]:
df[df.Set=='Test']

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
1,1.031741e-03,0.022531,0.481587,-755.943496,0.000000,-1824.017334,-9924.245758,GCN,Test,180
3,1.220445e-05,0.002801,0.163308,-3.028061,0.000000,-57.299070,-276.002546,GCN with transfer learning,Test,180
5,2.297604e-08,0.000073,0.003909,0.989085,1.000000,0.981748,0.959730,Random Forest,Test,180
7,1.824250e-04,0.010227,0.338595,-107.680914,0.000000,-410.452558,-2639.883435,GCN,Test,445
9,1.574461e-05,0.003146,0.196118,0.295736,0.383904,-72.745692,-407.906966,GCN with transfer learning,Test,445
11,8.390658e-09,0.000046,0.002416,0.995058,1.000000,0.992980,0.981827,Random Forest,Test,445
13,1.645967e-02,0.108885,0.843348,-6529.935799,0.000000,-65863.508984,-316316.061695,GCN,Test,9
15,2.301280e-05,0.003744,0.204735,-6.353722,0.000000,-61.949078,-495.073811,GCN with transfer learning,Test,9
17,4.054910e-07,0.000315,0.015321,0.697931,1.000000,0.698409,0.284956,Random Forest,Test,9
19,5.193004e-04,0.018067,0.504556,-68.214723,0.000000,-3902.891968,-34491.497420,GCN,Test,92


In [9]:
df = df.replace('Fresh', 'GCN')
df = df.replace('Transfer learning', 'GCN with transfer learning')

In [10]:
alt.Chart(df[df.Set=='Test']).mark_circle().encode(
    x=alt.X('Train Size:Q', scale=alt.Scale(type='log')),
    y=alt.Y('mse:Q', title='MSE', axis=alt.Axis(format='.1e')),
    color='Model',
    tooltip=['Model', 'mse']
).properties(width=400, height=200, title='Transfer learning - Tower')

alt.Chart(...)